### 제 2유형 연습하기. 팁 예측하기 (회귀)

In [2]:
import pandas as pd 
import numpy as np 

# 정규화
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# 데이터분리
from sklearn.model_selection import train_test_split

# 모델 선정
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

# 모델 성능 평가 
# 회귀 관련 성능 평가 
from sklearn.metrics import r2_score, mean_squared_error

# 분류 관련 성능 평가
from sklearn.metrics import f1_score, recall_score, precision_score,roc_auc_score,accuracy_score

In [3]:
import seaborn as sns
df = sns.load_dataset('tips')

x= df.drop(columns='tip')
y = df['tip']

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size =0.2, random_state=2024)
x_train= pd.DataFrame(x_train.reset_index())
x_test= pd.DataFrame(x_test.reset_index())
y_train = pd.DataFrame(y_train.reset_index())

x_train.rename(columns={'index':'cust_id'},inplace=True)
x_test.rename(columns={'index':'cust_id'},inplace=True)
y_train.columns=['cust_id','tip']

cust_id = x_test['cust_id'].copy()

#### 레스토랑의 tip 예측 문제
##### - 데이터의 결측치, 이상치, 변수에 대해 처리하고
##### - 회귀모델을 사용하여 Rsq, MSE 값을 산출하시오

데이터셋 설명    
- total_bill(총 청구액) : 손님의 식사 총 청구액
- tip(팁) : 팁의 양
- sex(성별) : 손님의 성별
- smoker(흡연자) : 손님의 흡연 여부 ("Yes" 또는 "No")
- day(요일) : 식사가 이루어진 요일
- time(시간) : 점심 또는 저녁 중 언제 식사가 이루어졌는지
- size(인원 수) : 식사에 참석한 인원수

#### 01. 데이터 탐색(EDA)

In [4]:
print(x_train.describe().T)
print(x_test.describe().T)
print(y_train.describe().T)

            count        mean        std   min    25%     50%      75%     max
cust_id     195.0  122.394872  70.734606  2.00  61.50  123.00  183.500  243.00
total_bill  195.0   19.754564   8.699479  3.07  13.38   17.59   24.535   48.33
size        195.0    2.589744   0.977002  1.00   2.00    2.00    3.000    6.00
            count        mean        std   min    25%     50%     75%     max
cust_id      49.0  117.938776  70.579567  0.00  56.00  115.00  172.00  239.00
total_bill   49.0   19.910816   9.763248  7.25  13.28   18.24   22.23   50.81
size         49.0    2.489796   0.844651  1.00   2.00    2.00    3.00    6.00
         count        mean        std  min   25%     50%      75%    max
cust_id  195.0  122.394872  70.734606  2.0  61.5  123.00  183.500  243.0
tip      195.0    2.946615   1.330591  1.0   2.0    2.71    3.525    9.0


In [5]:
print(x_train.info())
print(x_test.info())
print(y_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   cust_id     195 non-null    int64   
 1   total_bill  195 non-null    float64 
 2   sex         195 non-null    category
 3   smoker      195 non-null    category
 4   day         195 non-null    category
 5   time        195 non-null    category
 6   size        195 non-null    int64   
dtypes: category(4), float64(1), int64(2)
memory usage: 6.0 KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   cust_id     49 non-null     int64   
 1   total_bill  49 non-null     float64 
 2   sex         49 non-null     category
 3   smoker      49 non-null     category
 4   day         49 non-null     category
 5   time        49 non-null     category
 6   size   

In [6]:
print(x_train.isnull().sum())
print(x_test.isnull().sum())
print(y_train.isnull().sum())

cust_id       0
total_bill    0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64
cust_id       0
total_bill    0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64
cust_id    0
tip        0
dtype: int64


In [7]:
# 데이터 형태 확인하기
print(x_train.dtypes)

cust_id          int64
total_bill     float64
sex           category
smoker        category
day           category
time          category
size             int64
dtype: object


In [9]:
# 각 카테고리별 종류 확인 (x_train)
print(x_train['sex'].value_counts())
print(x_train['smoker'].value_counts())
print(x_train['day'].value_counts())
print(x_train['time'].value_counts())


Male      127
Female     68
Name: sex, dtype: int64
No     118
Yes     77
Name: smoker, dtype: int64
Sat     71
Sun     57
Thur    49
Fri     18
Name: day, dtype: int64
Dinner    140
Lunch      55
Name: time, dtype: int64


In [10]:
# 각 카테고리별 종류 확인 
print(x_test['sex'].value_counts())
print(x_test['sex'].value_counts())
print(x_test['sex'].value_counts())
print(x_test['sex'].value_counts())


Male      30
Female    19
Name: sex, dtype: int64
Male      30
Female    19
Name: sex, dtype: int64
Male      30
Female    19
Name: sex, dtype: int64
Male      30
Female    19
Name: sex, dtype: int64


In [12]:
# 연속값 - 이상치 확인 
# 범주형 데이터의 경우 include = 'category' | 'object' 
print(x_train.describe(include='category').T)
print(x_test.describe(include='category').T)
print('----------------------------------')
print(x_train.describe())
print(x_test.describe())

       count unique     top freq
sex      195      2    Male  127
smoker   195      2      No  118
day      195      4     Sat   71
time     195      2  Dinner  140
       count unique     top freq
sex       49      2    Male   30
smoker    49      2      No   33
day       49      4     Sun   19
time      49      2  Dinner   36
----------------------------------
          cust_id  total_bill        size
count  195.000000  195.000000  195.000000
mean   122.394872   19.754564    2.589744
std     70.734606    8.699479    0.977002
min      2.000000    3.070000    1.000000
25%     61.500000   13.380000    2.000000
50%    123.000000   17.590000    2.000000
75%    183.500000   24.535000    3.000000
max    243.000000   48.330000    6.000000
          cust_id  total_bill       size
count   49.000000   49.000000  49.000000
mean   117.938776   19.910816   2.489796
std     70.579567    9.763248   0.844651
min      0.000000    7.250000   1.000000
25%     56.000000   13.280000   2.000000
50%    115.

In [13]:
# y 데이터 분석
print(y_train.head())

   cust_id   tip
0      209  2.23
1       88  5.85
2        2  3.50
3      142  5.00
4      125  4.20


#### 02. 데이터 전처리

- 원핫 인코딩 

In [14]:
x_train = pd.get_dummies(x_train)
x_test= pd.get_dummies(x_test)

x_train.drop(columns = {'cust_id'},axis=1, inplace=True)
x_test.drop(columns={'cust_id'},axis=1,inplace=True)

In [16]:
print(x_train.head(3))
print(x_test.head(3))

   total_bill  size  sex_Male  sex_Female  smoker_Yes  smoker_No  day_Thur  \
0       12.76     2         0           1           1          0         0   
1       24.71     2         1           0           0          1         1   
2       21.01     3         1           0           0          1         0   

   day_Fri  day_Sat  day_Sun  time_Lunch  time_Dinner  
0        0        1        0           0            1  
1        0        0        0           1            0  
2        0        0        1           0            1  
   total_bill  size  sex_Male  sex_Female  smoker_Yes  smoker_No  day_Thur  \
0       29.03     3         1           0           0          1         0   
1       35.83     3         0           1           0          1         0   
2       50.81     3         1           0           1          0         0   

   day_Fri  day_Sat  day_Sun  time_Lunch  time_Dinner  
0        0        1        0           0            1  
1        0        1        0          

#### 03. 데이터 분리

In [ ]:
x_trian, x_val, y_train, y_val = train_test_split